
# Extracción de Datos de Propiedades Inmobiliarias en CDMX

## Descripción
En este notebook, se realiza la extracción de datos de propiedades inmobiliarias en diferentes alcaldías de la Ciudad de México desde el sitio web Lamudi. Se recopila información relevante de cada propiedad, como:

- **Ubicación:** Dirección de la propiedad.
- **Precio:** Precio de venta de la propiedad.
- **Descripción:** PDetalles sobre la propiedad.
- **Metros Cuadrados:** Área construida de la propiedad en metros cuadrados.
- **Número de Recámaras:** Cantidad de recámaras.
- **Número de Baños:** Cantidad de baños.

Estos datos se guardan en un archivo CSV para su posterior análisis y modelado.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Encabezado de agente de usuario
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_page_content(url):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Error {response.status_code} al obtener la página: {url}")
        return None

def extract_text_or_none(soup, selector, attribute=None):
    element = soup.select_one(selector)
    if element:
        if attribute:
            return element.get(attribute, None).strip()
        return element.text.strip()
    return None

def parse_property_data(page_content, alcaldia):
    soup = BeautifulSoup(page_content, 'html.parser')
    properties = []

    listings_container = soup.find('div', class_='listings__cards notSponsored', id='listings-content')

    if not listings_container:
        print("No se encontró el contenedor principal.")
        return properties

    for property_container in listings_container.find_all('div', class_='snippet__content'):
        title = extract_text_or_none(property_container, 'span.snippet__content__title')
        price = extract_text_or_none(property_container, 'div.snippet__content__price')
        location = extract_text_or_none(property_container, 'span[data-test="snippet-content-location"]')
        description = extract_text_or_none(property_container, 'div.snippet__content__description')
        bedrooms = extract_text_or_none(property_container, 'span.property__number.bedrooms')
        bathrooms = extract_text_or_none(property_container, 'span.property__number.bathrooms')
        area = extract_text_or_none(property_container, 'span.property__number.area')

        properties.append({
            'alcaldia': alcaldia,
            'title': title,
            'price': price,
            'location': location,
            'description': description,
            'bedrooms': bedrooms,
            'bathrooms': bathrooms,
            'area': area
        })

    return properties

def scrape_properties(base_url, alcaldia, num_pages):
    all_properties = []
    for page in range(1, num_pages + 1):
        url = f"{base_url}?page={page}"
        page_content = get_page_content(url)
        if page_content:
            properties = parse_property_data(page_content, alcaldia)
            all_properties.extend(properties)
        else:
            print(f"Error al obtener la página {page}")

    return all_properties

# Ejemplo de uso con múltiples alcaldías
base_urls = {
    'Álvaro Obregón': 'https://www.lamudi.com.mx/distrito-federal/alvaro-obregon/for-sale/',
    'Azcapotzalco': 'https://www.lamudi.com.mx/distrito-federal/azcapotzalco/for-sale/',
    'Benito Juárez': 'https://www.lamudi.com.mx/distrito-federal/benito-juarez/for-sale/',
    'Coyoacán': 'https://www.lamudi.com.mx/distrito-federal/coyoacan/for-sale/',
    'Cuauhtémoc': 'https://www.lamudi.com.mx/distrito-federal/cuauhtemoc-2/for-sale/',
    'Gustavo A. Madero': 'https://www.lamudi.com.mx/distrito-federal/gustavo-a-madero/for-sale/',
    'Miguel Hidalgo': 'https://www.lamudi.com.mx/distrito-federal/miguel-hidalgo/for-sale/',
}
num_pages = 7  # Número de páginas a scrapear

all_properties = []
for alcaldia, base_url in base_urls.items():
    print(f"Scrapeando propiedades en {alcaldia}")
    properties = scrape_properties(base_url, alcaldia, num_pages)
    all_properties.extend(properties)

# Guardar los datos en un DataFrame de pandas
df = pd.DataFrame(all_properties)
df.to_csv('departamentos_cdmx.csv', index=False)

print("Datos guardados en 'departamentos_cdmx.csv'")


Scrapeando propiedades en Álvaro Obregón
Scrapeando propiedades en Azcapotzalco
Scrapeando propiedades en Benito Juárez
Scrapeando propiedades en Coyoacán
Scrapeando propiedades en Cuauhtémoc
Scrapeando propiedades en Gustavo A. Madero
Scrapeando propiedades en Miguel Hidalgo
Datos guardados en 'departamentos_cdmx.csv'
